In [1]:
import os

import pysam
import pandas as pd

----

In [4]:
def count_rRNAs(alignment_file, iterator):
    
    rRNA_counts = 0

    for ch, start, end in iterator:
        
        for read in alignment_file.fetch(region=ch, start=start, end=end):
            
            nh = read.get_tag('NH')
            inc = 1 / nh
            rRNA_counts += inc
                
    return round(rRNA_counts,2)

In [5]:
def count_mapped(alignment_file):
    
    total = 0
    for read in alignment_file.fetch(until_eof=True):
            
        nh = read.get_tag('NH')
        inc = 1 / nh
        total += inc
                
    return round(total,2)

In [6]:
def bed_coverage_dict(in_bed):
    
    cov_dict = {}
    with open(in_bed) as itc:
        for line in itc:
            ls = line.split('\t')
            ch = ls[0].strip(' ')
            pos = ls[1].strip(' ')
            val = 0
            key = ch + ':' + pos
            cov_dict[key] = 0
            
    return cov_dict

In [7]:
def generate_coverage_dict(alignmentFile, in_bed):

    coverage_dict = bed_coverage_dict(in_bed)

    for read in alignmentFile.fetch(until_eof=True):

        nh = read.get_tag('NH')
        inc = 1.0 / nh
        ch = read.reference_name
        blocks = read.get_blocks()
        
        for start, end in blocks:
            update_coverage_dict(coverage_dict, ch, start + 1, end + 1, inc)
        
    return coverage_dict

In [8]:
def update_coverage_dict(coverage_dict, ch, start, end, inc):
    
    for i in range(start, end):
        key = ch + ':' + str(i)
        coverage_dict[key] += inc

In [9]:
def write_coverage_file(coverage_dict, out_file):
    
    with open(out_file, 'w+') as out:

        for coord, item in coverage_dict.items():

            ch = coord.split(':')[0]
            pos = coord.split(':')[1]
            line = ch + '\t' + str(pos) + '\t' + str(round(item, 2)) + '\n'
            out.write(line)    

-----

### Step 1 : calculate coverages for bam files

In [10]:
in_bed = '/data/parastou/RNAdeg/CoveragePlots/coverage_scaffold.txt'

In [11]:
bam_dir = '/data/parastou/RNAdeg/results/RipRna/coverage_bams/'

In [12]:
out_dir = '/data/parastou/RNAdeg/CoveragePlots/RNA/'

#### Load gene annotation data frame and generate rRNA iterator

In [13]:
in_gdf = pd.read_csv('/data/parastou/RNAdeg/annotation/schizosaccharomyces_pombe.chr.extended.csv', sep='\t')

In [14]:
rRNA_iter = in_gdf[in_gdf['bio_type']=='rRNA'][['chr', 'start', 'end']].values

#### Calculate coverages

In [ ]:
for file in os.listdir(bam_dir):
    
    if file.endswith('.bam'):
        
        base_name = file.split('.')[0]
        in_path = os.path.join(bam_dir, file)
        suffix = ''
        if 'forward' in file:
            suffix = '.forward'
        elif 'reverse' in file:
            suffix = '.reverse'

        out_path = os.path.join(out_dir, base_name + suffix + '.txt')
        out_norm_path = os.path.join(out_dir, base_name + suffix + '.norm.txt')

        print('bam file : %s ...' % in_path)

        ## Load a bam file and create coverage information for mapped regions
        st = pysam.AlignmentFile(open(in_path, 'rb'))      
        coverage_dict = generate_coverage_dict(st, in_bed)

        ## Merge coverage information into a new file
        write_coverage_file(coverage_dict, out_path)
        print('Coverage file saved in %s.' % out_path)

        ## Normalize coverages
        ## Calculate rRNA counts for this file
        st = pysam.AlignmentFile(open(in_path, 'rb'))  
        rrna_counts = count_rRNAs(st, rRNA_iter)
        print('Bam file contains %d alignments mapping to rRNAs.' % int(rrna_counts))

        ## Load coverage file and normalize
        df = pd.read_csv(out_path, sep='\t', names=range(3))
        st = pysam.AlignmentFile(open(in_path, 'rb')) 
        total = count_mapped(st)
        total -= rrna_counts
        scaling_factor = (total / 1000000)
        df[2] = round(df[2] / scaling_factor, 2)
        df.to_csv(out_norm_path, sep='\t', index=None, header=None)
        print('Normalized coverage file saved in %s.' % out_norm_path)

---